# Train and deploy with Cloud AI Platform

In [3]:
import json
import os
import pandas as pd
import time

## Create a training application

In [4]:
APP_FOLDER = '../training_app/trainer'
os.makedirs(APP_FOLDER, exist_ok=True)
!touch $APP_FOLDER/__init__.py

### Create a training script

In [5]:
%%writefile $APP_FOLDER/train.py

import logging
import os
import subprocess
import sys

import fire
import numpy as np
import pandas as pd

from sklearn.externals import joblib
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  param_grid = [
    {
      'reduce_dim__n_components': n_features_options,
      'regress__alpha': l2_reg_options
    }
  ]

  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)
  
  grid.fit(X, y)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/trained_model/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Writing ../training_app/trainer/train.py


### Configure dependencies

In [6]:
%%writefile $APP_FOLDER/../setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['fire', 'gcsfs']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My training application package.'
)


Writing ../training_app/trainer/../setup.py


## Submit a training job

In [7]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
REGION = "us-west1"
SCALE_TIER = "BASIC"
JOB_DIR = "gs://jk-demo-jobdir/{}".format(JOB_NAME)
MODULE_NAME = "trainer.train"
RUNTIME_VERSION = "1.14"
PYTHON_VERSION = "3.5"
DATA_PATH="gs://jk-demo-datasets/gasdata/training.csv"
N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOB_DIR \
--package-path $APP_FOLDER \
--module-name $MODULE_NAME \
--scale-tier $SCALE_TIER \
--python-version $PYTHON_VERSION \
--runtime-version $RUNTIME_VERSION \
-- \
--data_path $DATA_PATH \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20190820_044415] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20190820_044415

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20190820_044415
jobId: JOB_20190820_044415
state: QUEUED


In [ ]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2019-08-20T04:44:16Z'
etag: aEyqQoM0RWY=
jobId: JOB_20190820_044415
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/training.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://jk-demo-jobdir/JOB_20190820_044415
  packageUris:
  - gs://jk-demo-jobdir/JOB_20190820_044415/packages/1093509a83d6a14e1733e6e21af19171094ac979e6e37c9d093b2a16f93799fd/trainer-0.1.tar.gz
  pythonModule: trainer.train
  pythonVersion: '3.5'
  region: us-west1
  runtimeVersion: '1.14'
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20190820_044415?project=jk-demo1

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20190820_044415&project=jk-demo1
INFO	2019-08-20 04:44:17 +0000	service		Job JOB_20190820_044415 is queued.
INFO	2019-08-20 04:44:17 +0000	service		Waiting for job to be provisioned.
INFO	2019-08-20 0

## Deploy using AI Platform Prediction

### Create a model resource

In [9]:
MODEL_NAME = "gas_octane_regression"
REGION = "us-central1"
LABELS = "task=regression,domain=chemometrics"

!gcloud ai-platform models create  $MODEL_NAME \
--regions=$REGION \
--labels=$LABELS

Created ml engine model [projects/jk-demo1/models/gas_octane_regression].


### Create a model version

In [10]:
MODEL_PATH = "{}/trained_model/".format(JOB_DIR)
VERSION_NAME = "octane_v01"

!gcloud ai-platform versions create $VERSION_NAME \
--model=$MODEL_NAME \
--origin=$MODEL_PATH \
--runtime-version=1.14 \
--framework=scikit-learn \
--python-version=3.5

Creating version (this might take a few minutes)......done.                    


### Test the model
#### Prepare a file with test instances

In [11]:
TEST_DATASET_PATH = "gs://jk-demo-datasets/gasdata/testing.csv"
INPUT_FILE = 'instances.json'

df_test = pd.read_csv(TEST_DATASET_PATH).drop('octane', axis=1)
with open(INPUT_FILE, "w") as f:
  for index, row in df_test.iterrows():
    f.write(json.dumps(list(row.values)))
    f.write("\n")

#### Run predictions

In [12]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $VERSION_NAME \
--json-instances $INPUT_FILE

[83.61274117546486, 88.78144180442887, 86.60147562277632, 86.37821707761988, 84.57226241557935, 88.60871164935777]
